In [1]:
# Importing necessary libraries
import numpy as np

In [2]:
# 1
# Example dataset of gaming computers with their specifications
gaming_computers = {
    "PC1": {"price": 900, "performance": 85, "power_consumption": 65, "screen_size": 15.6, "ram": 8, "storage": 512},
    "PC2": {"price": 1200, "performance": 95, "power_consumption": 75, "screen_size": 17.3, "ram": 16, "storage": 1024},
    "PC3": {"price": 750, "performance": 80, "power_consumption": 55, "screen_size": 14, "ram": 8, "storage": 256},
    "PC4": {"price": 1500, "performance": 110, "power_consumption": 85, "screen_size": 18, "ram": 32, "storage": 2048},
    "PC5": {"price": 1100, "performance": 90, "power_consumption": 70, "screen_size": 16, "ram": 16, "storage": 1024},
}

# Weighting the importance of each attribute in gaming performance (for *Stalker 2* and similar games)
weights = {
    "price": 0.2,  # less important in gaming, but still a consideration
    "performance": 0.4,  # main factor for gaming
    "power_consumption": 0.1,  # energy efficiency is important, but secondary
    "screen_size": 0.1,  # useful but not as crucial as other factors
    "ram": 0.1,  # more RAM is good, but modern games also rely on CPU/GPU performance
    "storage": 0.1,  # SSD storage can improve game load times, but isn't as important as performance
}

def normalize(data, method="range"):
    """
    Normalize the data using the specified method.

    Args:
        data (dict): Raw data with computer configurations.
        method (str): The normalization method. Options: "range", "max".

    Returns:
        dict: Normalized data.
    """
    normalized = {}
    
    for indicator in data[next(iter(data))].keys():
        values = [alt[indicator] for alt in data.values()]
        min_val, max_val = min(values), max(values)
        
        for alt, metrics in data.items():
            if method == "range":
                norm_value = (metrics[indicator] - min_val) / (max_val - min_val)
            elif method == "max":
                norm_value = metrics[indicator] / max_val
            
            if alt not in normalized:
                normalized[alt] = {}
            normalized[alt][indicator] = norm_value

    return normalized

def additive_aggregation(normalized_data, weights):
    """
    Aggregate the normalized data using additive aggregation method.

    Args:
        normalized_data (dict): Normalized data.
        weights (dict): Weights for each attribute.

    Returns:
        dict: Aggregated scores for each computer.
    """
    aggregated = {}
    
    for alt, metrics in normalized_data.items():
        aggregated[alt] = sum(metrics[indicator] * weights[indicator] for indicator in metrics)
        
    return aggregated

def multiplicative_aggregation(normalized_data, weights):
    """
    Aggregate the normalized data using multiplicative aggregation method.

    Args:
        normalized_data (dict): Normalized data.
        weights (dict): Weights for each attribute.

    Returns:
        dict: Aggregated scores for each computer.
    """
    aggregated = {}
    
    for alt, metrics in normalized_data.items():
        aggregated[alt] = 1
        for indicator in metrics:
            aggregated[alt] *= metrics[indicator] ** weights[indicator]
            
    return aggregated

# Main process to normalize and aggregate data with different methods
def evaluate_computers(data, weights):
    """
    Evaluate the computers based on different aggregation methods.

    Args:
        data (dict): Raw data of computers.
        weights (dict): Weights for each attribute.

    Returns:
        dict: Final aggregated scores for each method.
    """
    results = {}

    # Range normalization + Additive aggregation
    normalized_data_range = normalize(data, method="range")
    results["Range + Additive"] = additive_aggregation(normalized_data_range, weights)

    # Range normalization + Multiplicative aggregation
    results["Range + Multiplicative"] = multiplicative_aggregation(normalized_data_range, weights)

    # Max normalization + Additive aggregation
    normalized_data_max = normalize(data, method="max")
    results["Max + Additive"] = additive_aggregation(normalized_data_max, weights)

    # Max normalization + Multiplicative aggregation
    results["Max + Multiplicative"] = multiplicative_aggregation(normalized_data_max, weights)

    return results

# Evaluate the computers using the defined methods
evaluation_results = evaluate_computers(gaming_computers, weights)

# Displaying the results
for method, result in evaluation_results.items():
    print(f"Results for {method}:")
    for computer, score in result.items():
        print(f"  {computer}: {score:.4f}")
    print()

Results for Range + Additive:
  PC1: 0.1943
  PC2: 0.5454
  PC3: 0.0000
  PC4: 1.0000
  PC5: 0.4029

Results for Range + Multiplicative:
  PC1: 0.0000
  PC2: 0.5306
  PC3: 0.0000
  PC4: 1.0000
  PC5: 0.3965

Results for Max + Additive:
  PC1: 0.6422
  PC2: 0.7898
  PC3: 0.5709
  PC4: 1.0000
  PC5: 0.7452

Results for Max + Multiplicative:
  PC1: 0.5923
  PC2: 0.7723
  PC3: 0.5060
  PC4: 1.0000
  PC5: 0.7319



In [3]:
# 2
def sugeno_integral(criteria, weights, density):
    """
    Calculates the Sugeno integral for a given set of criteria and weights.

    Args:
        criteria (list of float): The values of the criteria to aggregate.
        weights (list of float): The importance of each criterion (weights).
        density (function or list of float): A function or predefined density values to modify the aggregation process.

    Returns:
        float: The Sugeno integral aggregation result.
    
    Raises:
        ValueError: If the density values do not match the length of the criteria list.
    """
    # Sort the criteria and corresponding weights in descending order
    sorted_indices = np.argsort(criteria)[::-1]
    sorted_criteria = np.array(criteria)[sorted_indices]
    sorted_weights = np.array(weights)[sorted_indices]

    # Evaluate the density values
    if callable(density):
        density_values = [density(i + 1, len(criteria)) for i in range(len(criteria))]
    else:
        density_values = np.array(density)

    # Check if density values match the number of criteria
    if len(density_values) != len(criteria):
        raise ValueError("The density measure must match the number of criteria.")
    
    # Apply Sugeno integral aggregation: min(criterion, density) for each criterion, then max over all
    sugeno_values = [min(sorted_criteria[i], density_values[i]) for i in range(len(criteria))]
    
    return max(sugeno_values)


def normalize_data(raw_data):
    """
    Normalizes the raw data for each criterion based on the maximum value.

    Args:
        raw_data (dict): A dictionary where keys are PC names and values are dictionaries of criteria and their values.

    Returns:
        dict: A dictionary of normalized data for each PC.
    """
    normalized_data = {}
    
    for key, values in raw_data.items():
        normalized_data[key] = {
            criterion: (1 / value if criterion == "price" else value / max([d[criterion] for d in raw_data.values()]))
            for criterion, value in values.items()
        }
        
    return normalized_data


# Example dataset of gaming computers with their specifications
gaming_computers = {
    "PC1": {"price": 900, "performance": 85, "power_consumption": 65, "screen_size": 15.6, "ram": 8, "storage": 512},
    "PC2": {"price": 1200, "performance": 95, "power_consumption": 75, "screen_size": 17.3, "ram": 16, "storage": 1024},
    "PC3": {"price": 750, "performance": 80, "power_consumption": 55, "screen_size": 14, "ram": 8, "storage": 256},
    "PC4": {"price": 1500, "performance": 110, "power_consumption": 85, "screen_size": 18, "ram": 32, "storage": 2048},
    "PC5": {"price": 1100, "performance": 90, "power_consumption": 70, "screen_size": 16, "ram": 16, "storage": 1024},
}

# Weighting the importance of each attribute in gaming performance (for *Stalker 2* and similar games)
weights = {
    "price": 0.2,
    "performance": 0.4,
    "power_consumption": 0.1,
    "screen_size": 0.1,
    "ram": 0.1,
    "storage": 0.1,
}

# Normalize the data
normalized_data = normalize_data(gaming_computers)

# Modalities (density functions)
modalities = {
    "possibility": lambda i, n: 1,  # Maximum possibility, no change based on position
    "probability": lambda i, n: i / n,  # Linearly increasing density based on rank
    "necessity": lambda i, n: (n - i + 1) / n,  # Linearly decreasing density based on rank
}

# Results storage
results = {}

# Compute Sugeno integral for each modality
for modality, density_func in modalities.items():
    results[modality] = {}
    for pc, criteria_values in normalized_data.items():
        criteria_list = list(criteria_values.values())
        weight_list = list(weights.values())
        results[modality][pc] = sugeno_integral(criteria_list, weight_list, density_func)

# Display results
for modality, scores in results.items():
    print(f"Results for modality '{modality}':")
    for pc, score in scores.items():
        print(f"  {pc}: {score}")
    print()

Results for modality 'possibility':
  PC1: 0.8666666666666667
  PC2: 0.9611111111111111
  PC3: 0.7777777777777778
  PC4: 1.0
  PC5: 0.8888888888888888

Results for modality 'probability':
  PC1: 0.5
  PC2: 0.5
  PC3: 0.5
  PC4: 0.8333333333333334
  PC5: 0.5

Results for modality 'necessity':
  PC1: 0.8666666666666667
  PC2: 0.9611111111111111
  PC3: 0.7777777777777778
  PC4: 1.0
  PC5: 0.8888888888888888

